In [ ]:
#instalando pymongo
!pip install pymongo

In [1]:
#Biblioteca usada para requisitar uma pagina web
import urllib.request
import pandas as pd
from pymongo import MongoClient

In [2]:
# Definimos a url
# Verifique as permissões em https://www.python.org/robots.txt
with urllib.request.urlopen("https://www.worldfootball.net/scorer/eng-premier-league-2018-2019/") as url:
    page = url.read()

In [3]:
from bs4 import BeautifulSoup

In [4]:
# Analise o html na variável 'page' e armazene-o no formato Beautiful Soup
soup = BeautifulSoup(page, "html.parser")

In [5]:
soup.title

<title>Premier League 2018/2019 » Scorers</title>

In [ ]:
soup.find_all('table')

In [20]:
rodada = soup.find_all('option',{'class':"wahl"})[1]
rodada = rodada.get_text(strip=True).split('.')[0]

'1'

In [6]:
#selecionando tabela com a classe
Classif = soup.find_all('table',{'class':"standard_tabelle"})[0]

In [7]:
Cabecalho = Classif.find_all('tr')

In [8]:
header = Classif.find_all('th')

In [9]:
header_scorer = []
for cab in header:
    header_scorer.append(cab.get_text(strip=True))

In [8]:
header = ['Data','Hora','Home','campo1','Away','Placar','campo2','campo3']

In [10]:
jogos=[]
#pos = [0,1,2,4,5]
for jogo in Cabecalho:
    jogos.append([partida.get_text(strip=True) for partida in jogo.find_all('td')])
    
print (jogos)

[[], ['1.', 'Eden Hazard', '', 'Belgium', 'Chelsea FC', '10(7+3)'], ['2.', 'Kun Agüero', '', 'Argentina', 'Manchester City', '7(5+2)'], ['', 'Callum Wilson', '', 'England', 'AFC Bournemouth', '7(3+4)'], ['4.', 'Harry Kane', '', 'England', 'Tottenham Hotspur', '6(5+1)'], ['', 'Raheem Sterling', '', 'England', 'Manchester City', '6(4+2)'], ['', 'Alexandre Lacazette', '', 'France', 'Arsenal FC', '6(4+2)'], ['', 'Ryan Fraser', '', 'Scotland', 'AFC Bournemouth', '6(3+3)'], ['8.', 'Aleksandar Mitrović', '', 'Serbia', 'Fulham FC', '5(5+0)'], ['', 'Glenn Murray', '', 'England', 'Brighton & Hove Albion', '5(5+0)'], ['', 'Joshua King', '', 'Norway', 'AFC Bournemouth', '5(4+1)'], ['', 'Gylfi Sigurðsson', '', 'Iceland', 'Everton FC', '5(4+1)'], ['', 'Marko Arnautović', '', 'Austria', 'West Ham United', '5(4+1)'], ['', 'Pierre-Emerick Aubameyang', '', 'Gabon', 'Arsenal FC', '5(4+1)'], ['', 'James Maddison', '', 'England', 'Leicester City', '5(3+2)'], ['', 'Raúl Jiménez', '', 'Mexico', 'Wolverhampto

In [65]:
Partidas = pd.DataFrame.from_records(jogos,columns = header_scorer)
Partidas.dropna(inplace=True)

In [30]:
import re

In [66]:
Partidas['points'] = Partidas['points'].str.split('+')


In [44]:
ab = pd.Series(['a1', 'b2', 'c3'])
ab.str.extract('([ab])(\d)', expand=False)

,0,1
0,a,1
1,b,2
2,NaN,NaN


In [71]:
gols = [valor[0] for valor in Partidas['points']]
ass = [valor[1] for valor in Partidas['points']]
Partidas['gols'] = gols
Partidas['ass'] = ass


In [76]:
Partidas.head()

,#,Player,,Country,Team,points,gols,ass
1,1.,Eden Hazard,,Belgium,Chelsea FC,"[10(7, 3)]",10(7,3)
2,2.,Kun Agüero,,Argentina,Manchester City,"[7(5, 2)]",7(5,2)
3,,Callum Wilson,,England,AFC Bournemouth,"[7(3, 4)]",7(3,4)
4,4.,Harry Kane,,England,Tottenham Hotspur,"[6(5, 1)]",6(5,1)
5,,Raheem Sterling,,England,Manchester City,"[6(4, 2)]",6(4,2)


In [88]:
Partidas['ass'] = Partidas['ass'].str.split(')',expand=True)[0]

1      3
2      2
3      4
4      1
5      2
6      2
7      3
8      0
9      0
10     1
11     1
12     1
13     1
14     2
15     3
16     4
17     4
18     0
19     0
20     0
21     1
22     1
23     1
24     2
25     2
26     2
27     3
28     3
29     3
30     4
      ..
148    1
149    1
150    1
151    1
152    1
153    1
154    1
155    1
156    1
157    1
158    1
159    1
160    1
161    1
162    1
163    1
164    1
165    1
166    1
167    1
168    1
169    1
170    1
171    1
172    1
173    1
174    1
175    1
176    1
177    1
Name: 0, Length: 177, dtype: object

In [87]:
Partidas['gols'] = Partidas['gols'].str.split('(',expand=True)[1]


1      7
2      5
3      3
4      5
5      4
6      4
7      3
8      5
9      5
10     4
11     4
12     4
13     4
14     3
15     2
16     1
17     1
18     4
19     4
20     4
21     3
22     3
23     3
24     2
25     2
26     2
27     1
28     1
29     1
30     0
      ..
148    0
149    0
150    0
151    0
152    0
153    0
154    0
155    0
156    0
157    0
158    0
159    0
160    0
161    0
162    0
163    0
164    0
165    0
166    0
167    0
168    0
169    0
170    0
171    0
172    0
173    0
174    0
175    0
176    0
177    0
Name: 1, Length: 177, dtype: object

In [22]:
#Formata o placar
placar = [placar[0] for placar in Partidas['Placar'].str.split('(').tolist()]
Partidas['Placar'] = placar


,Data,Hora,Home,Away,Placar,Placar2
0,10/08/2018,20:00,Manchester United,Leicester City,2:1 (1:0),2:1
1,11/08/2018,12:30,Newcastle United,Tottenham Hotspur,1:2 (1:2),1:2
2,,15:00,AFC Bournemouth,Cardiff City,2:0 (1:0),2:0
3,,15:00,Fulham FC,Crystal Palace,0:2 (0:1),0:2
4,,15:00,Huddersfield Town,Chelsea FC,0:3 (0:2),0:3
5,,15:00,Watford FC,Brighton & Hove Albion,2:0 (1:0),2:0
6,,17:30,Wolverhampton Wanderers,Everton FC,2:2 (1:1),2:2
7,12/08/2018,13:30,Southampton FC,Burnley FC,0:0 (0:0),0:0
8,,13:30,Liverpool FC,West Ham United,4:0 (2:0),4:0
9,,16:00,Arsenal FC,Manchester City,0:2 (0:1),0:2


In [ ]:
#Selecionando cabecalho
head = []
for rows in Cabecalho:
    head.append(rows.get_text(strip=True))
    #print(type(rows))
    

In [ ]:
Classif.find_all('tr')

In [ ]:
#Selecionando dados para criacao do dataset
dados = []
for linha in Classif.find_all('tr')[1:]:
    dados.append([campo.get_text(strip=True) for campo in linha.find_all('td') ])
   

In [ ]:
dados_ok=[]
for d in dados:
    d.pop(1)
    dados_ok.append(d)

In [ ]:
Tabela_PremierLeague = pd.DataFrame.from_records(dados_ok,columns = head)

In [ ]:
Tabela_PremierLeague.columns

In [ ]:
Tabela_PremierLeague.rename(columns={'#':'rank','M.':'M','Dif.':'Dif','Pt.':'Pt'},inplace=True)

In [ ]:
#Cria coluna rodada
Tabela_PremierLeague['Rodada'] = rodada

In [50]:
conn = MongoClient('localhost',27017)

In [22]:
type(conn)

pymongo.mongo_client.MongoClient

In [51]:
#Criacao do Banco
db = conn.RankGeral

In [52]:
#Cricao da colecao
Premier_collection = db.PremierLeague_Table

In [ ]:
type(collection)

In [ ]:
PremierLeagueDict = Tabela_PremierLeague.to_dict('records')

In [ ]:
type(PremierLeagueDict)

In [ ]:
#inserindo dados na colecao
resultado = Premier_collection.insert_many(PremierLeagueDict)

In [ ]:
resultado.inserted_ids

In [53]:
#lista as colections do banco
db.list_collection_names()

['PremierLeague_Table']

In [25]:
#Exclui a collection
db.Tabelas.drop()

In [57]:
#lista primeiro registro
Premier_collection.find_one()

{'D': '0',
 'Dif': '7',
 'L': '0',
 'M': '2',
 'Pt': '6',
 'Rodada': '2',
 'Team': 'Manchester City',
 'W': '2',
 '_id': ObjectId('5bb952de81a0d535e12db2be'),
 'goals': '8:1',
 'rank': '1'}

In [ ]:
#lista todos os registros
for valor in Premier_collection.find():
    print(valor)

In [63]:
#Lista documentos
Premier_collection.estimated_document_count()

0

In [61]:
Premier_collection.find_one()

In [62]:
#remove todos os documentos
Premier_collection.delete_many({})

In [63]:
#Lista documentos
for rodadas in Premier_collection.find({'M':{'$gt':'1'}}):
    print(rodadas)

{'_id': ObjectId('5bb01ebe81a0d515c1895e18'), 'rank': '1', 'Team': 'Manchester City', 'M': '2', 'W': '2', 'D': '0', 'L': '0', 'goals': '8:1', 'Dif': '7', 'Pt': '6'}
{'_id': ObjectId('5bb01ebe81a0d515c1895e19'), 'rank': '2', 'Team': 'Liverpool FC', 'M': '2', 'W': '2', 'D': '0', 'L': '0', 'goals': '6:0', 'Dif': '6', 'Pt': '6'}
{'_id': ObjectId('5bb01ebe81a0d515c1895e1a'), 'rank': '3', 'Team': 'Chelsea FC', 'M': '2', 'W': '2', 'D': '0', 'L': '0', 'goals': '6:2', 'Dif': '4', 'Pt': '6'}
{'_id': ObjectId('5bb01ebe81a0d515c1895e1b'), 'rank': '4', 'Team': 'Watford FC', 'M': '2', 'W': '2', 'D': '0', 'L': '0', 'goals': '5:1', 'Dif': '4', 'Pt': '6'}
{'_id': ObjectId('5bb01ebe81a0d515c1895e1c'), 'rank': '5', 'Team': 'Tottenham Hotspur', 'M': '2', 'W': '2', 'D': '0', 'L': '0', 'goals': '5:2', 'Dif': '3', 'Pt': '6'}
{'_id': ObjectId('5bb01ebe81a0d515c1895e1d'), 'rank': '6', 'Team': 'AFC Bournemouth', 'M': '2', 'W': '2', 'D': '0', 'L': '0', 'goals': '4:1', 'Dif': '3', 'Pt': '6'}
{'_id': ObjectId('5bb